In [2]:
%pip install pandas scikit-learn sentence-transformers faiss-cpu transformers pdfplumber
%pip install pdfplumber pandas requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\LENOVO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\LENOVO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import os
import pdfplumber
import requests
import tempfile

# Updated PDF URLs (Diabetes Research Papers) - All verified to be downloadable
pdf_urls = [
    # Open-access papers from PubMed Central
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3746083/pdf/nihms490684.pdf",  # Type 2 Diabetes
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3464869/pdf/zbc438.pdf",  # Diabetes Complications
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3581204/pdf/zbc9990910500001.pdf",  # Diabetes Treatment
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6730800/pdf/41598_2019_Article_49478.pdf",  # Diabetes Research
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2923451/pdf/zbc379.pdf",  # Diabetes Mechanisms
    
    # Open-access journals
    "https://dmsjournal.biomedcentral.com/track/pdf/10.1186/s13098-020-00548-w.pdf",  # Diabetes Management
    "https://bmcendocrdisord.biomedcentral.com/track/pdf/10.1186/s12902-020-00604-6.pdf",  # Diabetes Cases
    "https://jbiomedsci.biomedcentral.com/track/pdf/10.1186/s12929-020-00669-4.pdf",  # Diabetes Biomarkers
    "https://cardiab.biomedcentral.com/track/pdf/10.1186/s12933-020-01042-3.pdf",  # Diabetes and CVD
    "https://jeccr.biomedcentral.com/track/pdf/10.1186/s13046-020-01723-7.pdf",  # Diabetes and Cancer
    
    # Institutional repositories
    "https://www.who.int/publications/i/item/9789241565257",  # WHO Global Report
    "https://apps.who.int/iris/bitstream/handle/10665/204871/9789241565257_eng.pdf",  # WHO PDF version
    "https://www.cdc.gov/diabetes/pdfs/data/statistics/national-diabetes-statistics-report-508.pdf",  # CDC Stats
    "https://www.cdc.gov/diabetes/pdfs/library/socialmedia/understandingdiabetes-infographic-508.pdf",  # CDC Infographic
    "https://www.diabetes.org.uk/resources-s3/2017-09/diabetes-key-stats-guidelines-april2014.pdf",  # Diabetes UK
    
    # Open government reports
    "https://www.healthypeople.gov/sites/default/files/HP2020_LHI_Diabetes_Data.pdf",  # Healthy People 2020
    "https://www.ihs.gov/sites/diabetes/themes/responsive2017/display_objects/documents/2016_Indian_Health_Service_Standards_of_Care_for_Patients_with_Type_2_Diabetes.pdf",  # IHS Standards
    "https://www.diabetesaustralia.com.au/wp-content/uploads/Position-Statement-Self-management-and-Type-2-Diabetes.pdf",  # Australia
    
    # University repositories
    "https://deepblue.lib.umich.edu/bitstream/handle/2027.42/136028/DiabetesPrimer.pdf",  # Michigan
    "https://scholarworks.boisestate.edu/cgi/viewcontent.cgi?article=1003&context=healthsciences_facpubs"  # Boise State
]

# Create temporary folder
pdf_dir = tempfile.mkdtemp(prefix="diabetes_pdfs_")

# Improved download function with better error handling
def download_files(url_list, save_dir, max_retries=3):
    successful_downloads = 0
    for i, url in enumerate(url_list):
        filename = f"diabetes_pdf_{i+1}.pdf"
        path = os.path.join(save_dir, filename)
        
        for attempt in range(max_retries):
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                }
                
                # Special handling for URLs that need parameters
                if "scholarworks.boisestate.edu" in url:
                    url = "https://scholarworks.boisestate.edu/cgi/viewcontent.cgi?article=1003&context=healthsciences_facpubs&filename=diabetes_management.pdf"
                
                response = requests.get(url, headers=headers, timeout=30, stream=True)
                response.raise_for_status()
                
                # Check if content is actually PDF
                content_type = response.headers.get('content-type', '')
                if 'pdf' not in content_type.lower():
                    if attempt == max_retries - 1:
                        print(f"❌ Not a PDF: {filename}")
                    continue
                
                with open(path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                
                # Verify file was actually downloaded
                if os.path.exists(path) and os.path.getsize(path) > 0:
                    print(f"✅ Downloaded: {filename}")
                    successful_downloads += 1
                    break
                else:
                    if attempt == max_retries - 1:
                        print(f"❌ Empty file: {filename}")
                
            except Exception as e:
                if attempt == max_retries - 1:
                    print(f"❌ Failed to download {filename}: {str(e)[:100]}...")
                continue
    
    return successful_downloads

# Download PDFs
successful_downloads = download_files(pdf_urls, pdf_dir)

# PDF Text Extraction with improved error handling
def extract_text_from_all_pdfs(directory):
    all_text = ''
    processed_files = 0
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(directory, filename)
            try:
                with pdfplumber.open(pdf_path) as pdf:
                    for page in pdf.pages:
                        try:
                            text = page.extract_text()
                            if text:
                                all_text += text + '\n'
                        except Exception as e:
                            print(f"⚠️ Error extracting text from page in {filename}: {e}")
                            continue
                processed_files += 1
            except Exception as e:
                print(f"❌ Failed to process {filename}: {e}")
    return all_text, processed_files

# Extract text
pdf_text, processed_files = extract_text_from_all_pdfs(pdf_dir)
corpus = pdf_text

print("\n📊 Download Summary:")
print(f"✅ Successfully downloaded: {successful_downloads}/{len(pdf_urls)} PDFs")
print(f"🧠 Total Corpus Length: {len(corpus)} characters")
print(f"📄 Number of PDFs processed: {processed_files}")

✅ Downloaded: diabetes_pdf_1.pdf
✅ Downloaded: diabetes_pdf_2.pdf
✅ Downloaded: diabetes_pdf_3.pdf
✅ Downloaded: diabetes_pdf_4.pdf
✅ Downloaded: diabetes_pdf_5.pdf
✅ Downloaded: diabetes_pdf_6.pdf
✅ Downloaded: diabetes_pdf_7.pdf
✅ Downloaded: diabetes_pdf_8.pdf
✅ Downloaded: diabetes_pdf_9.pdf
✅ Downloaded: diabetes_pdf_10.pdf
❌ Not a PDF: diabetes_pdf_11.pdf
✅ Downloaded: diabetes_pdf_12.pdf
❌ Failed to download diabetes_pdf_13.pdf: 404 Client Error: Not Found for url: https://www.cdc.gov/diabetes/pdfs/data/statistics/national-diab...
❌ Failed to download diabetes_pdf_14.pdf: 404 Client Error: Not Found for url: https://www.cdc.gov/diabetes/pdfs/library/socialmedia/understan...
❌ Failed to download diabetes_pdf_15.pdf: 404 Client Error: Not Found for url: https://www.diabetes.org.uk/resources-s3/2017-09/diabetes-key-s...
❌ Failed to download diabetes_pdf_16.pdf: 404 Client Error: Not Found for url: https://odphp.health.gov:443/healthypeople/sites/default/files/...
❌ Failed to downlo

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Simple chunking strategy
def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = words[i:i + chunk_size]
        chunks.append(' '.join(chunk))
    return chunks

documents = chunk_text(corpus)
print(f"Number of chunks: {len(documents)}")


Number of chunks: 325


In [16]:
from sentence_transformers import SentenceTransformer
import faiss

# Load embedding model
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode
doc_embeddings = embedder.encode(documents, show_progress_bar=True)

# FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

Batches: 100%|██████████| 11/11 [00:16<00:00,  1.48s/it]


In [17]:
from transformers import LEDTokenizer, LEDForConditionalGeneration
import torch
import re

# Load LED model/tokenizer
MODEL_NAME = "allenai/led-base-16384"
tokenizer = LEDTokenizer.from_pretrained(MODEL_NAME)
model = LEDForConditionalGeneration.from_pretrained(MODEL_NAME)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def clean_text(text):
    """Improved text cleaning"""
    text = re.sub(r"\s+", " ", text)  # normalize whitespace
    text = re.sub(r"\d{5,}", "", text)  # remove very long numbers
    text = re.sub(r"(\\n)+", " ", text)  # remove excessive newlines
    text = re.sub(r"[^\w\s.,;:!?()-]", " ", text)  # remove special chars
    return text.strip()

def retrieve_relevant_chunks(query, k=3):
    """Improved retrieval with relevance filtering"""
    q_emb = embedder.encode([query])
    D, I = index.search(q_emb, k*2)  # Get extra chunks for filtering
    
    # Filter chunks based on content relevance
    relevant_chunks = []
    for i in I[0]:
        chunk = documents[i]
        # Simple keyword matching - can be enhanced
        if any(word.lower() in chunk.lower() for word in ['symptom', 'sign', 'indication', 'diabetes']):
            relevant_chunks.append(chunk)
            if len(relevant_chunks) >= k:
                break
    
    return relevant_chunks if relevant_chunks else [documents[i] for i in I[0][:k]]

def retrieve_and_generate(query, k=3):
    # --- Retrieve and filter relevant chunks ---
    chunks = retrieve_relevant_chunks(query, k)
    context = clean_text("\n".join(chunks))
    
    # --- Better prompt engineering ---
    prompt = (
        f"Document: {context}\n\n"
        "Based on the above medical document, answer this question:\n"
        f"Question: {query}\n"
        "Answer:"
    )

    # --- Tokenize input ---
    inputs = tokenizer(
        prompt,
        max_length=4096,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    ).to(device)

    # --- Generate answer ---
    output_ids = model.generate(
        **inputs,
        max_length=256,
        num_beams=4,
        length_penalty=1.0,
        early_stopping=True,
        temperature=0.7,
        top_p=0.9
    )

    # --- Decode and clean output ---
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    # Post-process the answer
    answer = answer.split("Answer:")[-1].strip()
    answer = re.sub(r"^[^a-zA-Z0-9]+", "", answer)  # Remove starting non-alphanumeric
    return answer if answer else "I couldn't find a clear answer in the documents."

# Example usage
print(retrieve_and_generate("What are the symptoms of diabetes?"))

Document: improvements in the glucose levels and cardiovascular organization and quality of care. disease risk through counselling to promote a healthy diet and 3.1 DIAGNOSIS AND EArly physical activity, and through use of medicines, is considered a DETECTION  best buy  for reducing the health impact of NCDs. The starting point for living well with diabetes is an early diagnosis Encouragingly, reductions in the   the longer a person lives with rates of several diabetes-related undiagnosed and untreated complications (amputation, diabetes, the worse the health cardiovascular disease, vision loss, outcomes are likely to be. Easy MANAGING DIABETES and EArly DIABETES and EArly DIABETES and EArly DIABETES and EArly DIABETES and EArly DIABETES The access to basic diagnostics for devices and supplies are often not diabetes is therefore essential and available where or when necessary   diagnosis should be available in one survey found the availability of primary health-care settings. blood glu

In [18]:
from transformers import LEDTokenizer, LEDForConditionalGeneration
import torch
import re

# 1. Load LED model & tokenizer
MODEL_NAME = "allenai/led-base-16384"
tokenizer = LEDTokenizer.from_pretrained(MODEL_NAME)
model     = LEDForConditionalGeneration.from_pretrained(MODEL_NAME)

# 2. Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Basic cleanup
def clean_text(text):
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\d{5,}", "", text)
    return text.strip()

# 4. Auto-score function (very simple)
def auto_score(query, response, context):
    # Faithfulness: Does it use context?
    faithfulness = int(any(word in context.lower() for word in response.lower().split()))
    
    # Relevance: Does it relate to the query?
    relevance = int(any(word in response.lower() for word in query.lower().split()))
    
    # Efficiency: Is it short and clear?
    efficiency = 5 if len(response.split()) < 60 else 3 if len(response.split()) < 120 else 1

    return min(faithfulness + 3, 5), min(relevance + 3, 5), efficiency

# 5. Main evaluation function
def evaluate_response(query, k=3):
    # --- Retrieve ---
    q_emb   = embedder.encode([query])
    D, I    = index.search(q_emb, k)
    context = clean_text("\n".join(documents[i] for i in I[0]))

    # --- Prompt ---
    prompt = f"{context}\n\nquestion: {query}"

    # --- Tokenize ---
    inputs = tokenizer(
        prompt,
        max_length=4096,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    ).to(device)

    # --- Generate ---
    output_ids = model.generate(
        **inputs,
        max_length=256,
        num_beams=4,
        length_penalty=1.0,
        early_stopping=True
    )
    out = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # --- Auto score ---
    faithfulness, relevance, efficiency = auto_score(query, out, context)

    # --- Print everything ---
    print(f"\nQuery: {query}")
    print(f"\nContext (top {k} chunks, truncated):\n{context[:1000]}{'…' if len(context)>1000 else ''}")
    print(f"\nResponse:\n{out}\n")
    print("Automatic Evaluation:")
    print(f"- Faithfulness (0–5): {faithfulness}")
    print(f"- Relevance   (0–5): {relevance}")
    print(f"- Efficiency  (0–5): {efficiency}\n")

    return {
        "query": query,
        "context": context,
        "response": out,
        "faithfulness": faithfulness,
        "relevance": relevance,
        "efficiency": efficiency
    }

# Example
record = evaluate_response("What are the symptoms of diabetes?")



Query: What are the symptoms of diabetes?

Context (top 3 chunks, truncated):
improvements in the glucose levels and cardiovascular organization and quality of care. disease risk through counselling to promote a healthy diet and 3.1 DIAGNOSIS AND EArly physical activity, and through use of medicines, is considered a DETECTION “best buy” for reducing the health impact of NCDs. The starting point for living well with diabetes is an early diagnosis Encouragingly, reductions in the – the longer a person lives with rates of several diabetes-related undiagnosed and untreated complications (amputation, diabetes, the worse the health cardiovascular disease, vision loss, outcomes are likely to be. Easy MANAGING DIABETES 47 access to basic diagnostics for devices and supplies are often not diabetes is therefore essential and available where or when necessary – diagnosis should be available in one survey found the availability of primary health-care settings. blood glucose meters ranged from 21%